# preprocess

In [ ]:
import sys
import os
import logging as log
from pathlib import Path

import numpy as np
import scipy as sp
import pandas as pd

import scanpy as sc
import loompy as lp

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

from scanpy_helpers import get_metacells

In [ ]:
log.basicConfig(level=log.INFO)
sc.settings.njobs = 8

## params

In [ ]:
# input
load_anndata = "anndata.h5ad"

# output
output_dir = "outputs/prepare_scenic/"
f_loom_path_scenic = "outputs/prepare_scenic/filtered_scenic.loom"
cellt_map = "barcode_celltype_map.csv"

# params
annot_column = "ann_vas_2"
cell_types = ['capillary EC', 'venous EC', 'arterial EC', 'endocardial EC', 'littoral cells', 'lymphatic EC', 'pericytes', 'smooth muscle']  # subset to specific cell types

## 1) Load data

### single cell dataset

In [ ]:
log.info(f"load anndata: {load_anndata}")

ad = sc.read_h5ad(load_anndata)

In [ ]:
if ad.raw:
    ad = ad.raw.to_adata()
elif ad.X.shape[1] < 15000:
    raise ValueError("anndata genes dimension seems to be filtered")

ad.raw = ad

save var names to gene symbols

In [ ]:
ad_sub.var["gene_symbol"] = ad_sub.var_names.tolist()
ad_sub.var = ad_sub.var.set_index("gene_symbol")

#### UMAP

In [ ]:
log.info("plot UMAP")

try:
    with plt.rc_context({"figure.figsize": (15,15), "figure.dpi": 300, "figure.frameon": False}):
        sc.pl.umap(
            ad,
            color=annot_column,
            alpha=0.7,
            size=50,
            add_outline=True,
            outline_width = (0.25, 2.5),
            legend_fontoutline=3,
            legend_loc="on data",
            title = ""
        )
except:
    pass

## 2) Subset data

### subset cells

In [ ]:
log.info("subset cells")

In [ ]:
if cell_types:
    ad_sub = ad[ad.obs[annot_column].isin(cell_types)]
else:
    ad_sub = ad

### filtering

In [ ]:
sc.pp.filter_cells(ad_sub, min_genes = 100)
sc.pp.filter_genes(ad_sub, min_cells=3)

ad_sub = ad_sub[ad_sub.obs['n_genes'] < 7000, :]
ad_sub = ad_sub[ad_sub.obs['percent_mito'] < 0.15, :]

### save barcode celltype mapping

In [ ]:
ad_sub.obs["celltype_annotation"] = ad_sub.obs[annot_column]
ad_sub.obs["celltype_annotation"].to_csv(cellt_map)

## 4) Create metacells

### compute connectivities

In [ ]:
log.info("compute connectivities")

In [ ]:
ad_sub.raw = ad_sub

In [ ]:
# fix an error where 'log1p' entry missing in anndata object
if "log1p" not in ad_sub.uns:
    log.warning("'log1p' not in adata.uns")
    ad_sub.uns["log1p"] = {}
if "base" not in ad_sub.uns["log1p"]:
    log.warning("'base' of log not stored in adata.uns")
    ad_sub.uns["log1p"]["base"] = None

In [ ]:
sc.pp.highly_variable_genes(ad_sub)

sc.pp.scale(ad_sub)
sc.tl.pca(ad_sub)

sc.pp.neighbors(ad_sub)

In [ ]:
sc.tl.umap(ad_sub)

In [ ]:
sc.pl.umap(ad_sub, color = [annot_column, batch_column] if batch_column else annot_column)

In [ ]:
ad_sub.X = ad_sub.raw.X

### determine pseudobulk size

In [ ]:
n_cells = ad_sub.X.shape[0]

pseudobulk_size = 0
target_ncells = 10000
extend_pseudobulk_size_up_to = 25
subsample = 5000

if target_ncells and not pseudobulk_size:
    pseudobulk_size = round(n_cells / target_ncells)
    
    if subsample > 1 and subsample < target_ncells:
        # make larger pseudobulks instead of subsampling if possible
        larger_pseudobulk_size = round(n_cells / subsample)
        if larger_pseudobulk_size < extend_pseudobulk_size_up_to:
            pseudobulk_size = larger_pseudobulk_size
            subsample = 0

### make metacells

In [ ]:
log.info("compute mini-pseudobulks")

ad_sub = get_metacells(ad_sub, max_group_size = pseudobulk_size)

sc.pp.calculate_qc_metrics(ad_sub, log1p=False, inplace=True)

### Subsample

In [ ]:
if subsample:
    if subsample > 1:
        # by total number
        ad_sub = sc.pp.subsample(ad_sub, n_obs=int(subsample), copy=True)
    else:
        # by fraction
        ad_sub = sc.pp.subsample(ad_sub, fraction=float(subsample), copy=True)

### Normalise

In [ ]:
ad_sub.raw = ad_sub

sc.pp.highly_variable_genes(ad_sub)

sc.pp.scale(ad_sub)
sc.tl.pca(ad_sub)

sc.pp.neighbors(ad_sub)

In [ ]:
sc.tl.umap(ad_sub)

In [ ]:
sc.pl.umap(ad_sub, color = annot_column)

## 5) Save loom file

Output the basic filtered expression matrix to a loom file.  

This can also be used in the command-line pySCENIC steps

In [ ]:
log.info("save to loom file")

# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(ad_sub.var_names) ,
}
col_attrs = {
    "CellID": np.array(ad_sub.obs_names) ,
    "nGene": np.array( np.sum(ad_sub.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(ad_sub.X.transpose() , axis=0)).flatten() ,
}
lp.create( str(f_loom_path_scenic), ad_sub.X.transpose(), row_attrs, col_attrs)

In [ ]:
log.info("done.")